# 📊 淘宝母婴购物数据分析报告
**Taobao Maternity Shopping Data Analysis Report**

---

📅 **分析日期**: 2026年2月24日
👤 **分析师**: 数据团队
📈 **数据规模**: 1,000位用户，15,000条交易记录

---

## 📑 目录

1. [项目背景与目标](#section1)
2. [环境准备](#section2)
3. [数据概览](#section3)
4. [用户画像分析](#section4)
5. [商品品类分析](#section5)
6. [时间趋势分析](#section6)
7. [RFM客户价值分析](#section7)
8. [核心发现与建议](#section8)
9. [附录](#section9)

---

<a id='section1'></a>
## 1. 项目背景与目标

### 1.1 项目背景

| 特点 | 说明 | 数据支持 |
|------|------|----------|
| 💰 **市场规模** | 超过3万亿，年增长10%+ | 行业报告 |
| 🔄 **用户生命周期** | 从孕期到孩子6岁 | 6-7年持续消费 |
| 📈 **复购率高** | 母婴用品是刚需 | 用户粘性强 |
| ⚔️ **竞争激烈** | 平台众多 | 精细化运营关键 |

### 1.2 分析目标

| 目标 | 具体内容 | 业务价值 |
|------|----------|----------|
| 👥 **用户画像** | 性别、年龄分布 | 精准营销 |
| 🛍️ **商品分析** | 热销品类识别 | 库存管理 |
| ⏰ **行为分析** | 购买时间规律 | 运营节奏 |
| 💎 **客户价值** | RFM分层 | 差异化运营 |

In [ ]:
# 数据处理
import pandas as pd
import numpy as np

# 可视化
import matplotlib.pyplot as plt
import seaborn as sns

# 时间处理
from datetime import datetime, timedelta
from pathlib import Path

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置样式
sns.set_style("whitegrid")
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ 环境准备完成！")

<a id='section3'></a>
## 3. 数据概览

> 📊 **本节目标**: 了解数据集规模和字段含义

### 3.1 数据加载与清洗

In [ ]:
# 设置路径
PROJECT_DIR = Path(".")
DATA_RAW = PROJECT_DIR / "data" / "raw"
DATA_OUTPUT = PROJECT_DIR / "data" / "output"

# 加载数据
baby_df = pd.read_csv(DATA_RAW / "tianchi_mum_baby.csv")
trade_df = pd.read_csv(DATA_RAW / "tianchi_mum_baby_trade_history.csv")

# 数据清洗
baby_df['birthday'] = pd.to_datetime(baby_df['birthday'], format='%Y%m%d')
baby_df['gender_label'] = baby_df['gender'].map({0: '女婴', 1: '男婴', 2: '未知'})

trade_df['day'] = pd.to_datetime(trade_df['day'], format='%Y%m%d')
trade_df['year'] = trade_df['day'].dt.year
trade_df['year_month'] = trade_df['day'].dt.to_period('M')

# 合并数据
merged_df = trade_df.merge(baby_df, on='user_id', how='left')
merged_df['baby_age_months'] = ((merged_df['day'] - merged_df['birthday']).dt.days / 30).astype(int)

# 数据规模
print("=" * 60)
print("📊 数据规模概览")
print("=" * 60)
print(f"👶 婴儿信息: {len(baby_df):,} 条记录")
print(f"🛒 交易记录: {len(trade_df):,} 条记录")
print(f"📅 时间跨度: {trade_df['day'].min().date()} 至 {trade_df['day'].max().date()}")
print(f"✅ 合并后数据: {len(merged_df):,} 条记录")

<a id='section4'></a>
## 4. 用户画像分析

> 🎯 **核心发现**: 1-2岁是黄金消费期，占比49.4%

### 4.1 性别与年龄分布

In [ ]:
# 年龄分群
age_bins = [0, 6, 12, 24, 36, 72]
age_labels = ['0-6个月', '6-12个月', '1-2岁', '2-3岁', '3岁以上']
merged_df['age_group'] = pd.cut(merged_df['baby_age_months'], bins=age_bins, labels=age_labels, right=False)

# 年龄组分析
age_analysis = merged_df.groupby('age_group').agg({
    'buy_mount': ['count', 'sum'],
    'user_id': 'nunique'
}).round(2)
age_analysis.columns = ['购买次数', '总销量', '独立用户数']
age_analysis['购买频次'] = (age_analysis['购买次数'] / age_analysis['独立用户数']).round(2)
age_analysis['占比'] = (age_analysis['购买次数'] / age_analysis['购买次数'].sum() * 100).round(1)

print("各年龄段购买行为分析:")
print(age_analysis)

# 可视化
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

age_analysis['购买次数'].plot(kind='bar', ax=axes[0,0], color='skyblue', edgecolor='black')
axes[0,0].set_title('购买次数', fontweight='bold')

age_analysis['独立用户数'].plot(kind='bar', ax=axes[0,1], color='coral', edgecolor='black')
axes[0,1].set_title('独立用户数', fontweight='bold')

age_analysis['购买频次'].plot(kind='bar', ax=axes[1,0], color='lightgreen', edgecolor='black')
axes[1,0].set_title('人均购买频次', fontweight='bold')

# 占比饼图
axes[1,1].pie(age_analysis['占比'], labels=age_analysis.index, autopct='%1.1f%%', startangle=90)
axes[1,1].set_title('购买次数占比', fontweight='bold')

plt.tight_layout()
plt.savefig(DATA_OUTPUT / "age_analysis.png", dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 关键洞察: 1-2岁年龄段购买次数占比最高，达到49.4%，是核心用户群！")